# End-to-End Trading Project
## Часть 8: Production API и MLOps

### В этом ноутбуке:

1. **FastAPI сервис** для real-time прогнозов
2. **Model Registry** и версионирование
3. **Мониторинг** производительности
4. **Data Drift Detection**
5. **CI/CD Pipeline** (концепт)

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import joblib
import json
from datetime import datetime
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
print('Библиотеки загружены')

## 1. FastAPI Prediction Service

Создаём API для получения торговых сигналов в реальном времени.

In [ ]:
# Код FastAPI сервиса (сохраним как файл)

fastapi_code = '''
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import numpy as np
import joblib
import json
from typing import List, Dict

app = FastAPI(title="Trading Signal API", version="1.0")

# Загружаем модель и конфиг
model = joblib.load("models/lightgbm.joblib")
scaler = joblib.load("models/scaler.joblib")
with open("data/feature_sets.json") as f:
    config = json.load(f)

class PredictionRequest(BaseModel):
    features: Dict[str, float]

class PredictionResponse(BaseModel):
    probability: float
    signal: str
    confidence: str
    timestamp: str

@app.get("/health")
def health_check():
    return {"status": "healthy", "model_version": "1.0"}

@app.post("/predict", response_model=PredictionResponse)
def predict(request: PredictionRequest):
    try:
        # Извлекаем признаки
        feature_cols = config["extended_features"]
        features = [request.features.get(f, 0) for f in feature_cols]
        X = np.array(features).reshape(1, -1)
        
        # Нормализация
        X_scaled = scaler.transform(X)
        
        # Прогноз
        proba = model.predict_proba(X_scaled)[0, 1]
        
        # Генерация сигнала
        if proba > 0.6:
            signal, confidence = "BUY", "HIGH"
        elif proba > 0.55:
            signal, confidence = "BUY", "MEDIUM"
        elif proba < 0.4:
            signal, confidence = "SELL", "HIGH"
        elif proba < 0.45:
            signal, confidence = "SELL", "MEDIUM"
        else:
            signal, confidence = "HOLD", "LOW"
        
        return PredictionResponse(
            probability=float(proba),
            signal=signal,
            confidence=confidence,
            timestamp=datetime.now().isoformat()
        )
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

@app.get("/model/info")
def model_info():
    return {
        "model_type": "LightGBM",
        "num_features": len(config["extended_features"]),
        "features": config["extended_features"]
    }
'''

# Сохраняем код
with open('api_service.py', 'w') as f:
    f.write(fastapi_code)

print('FastAPI код сохранён в api_service.py')
print('\nЗапуск: uvicorn api_service:app --reload')
print('Docs: http://localhost:8000/docs')

## 2. Model Registry

In [ ]:
class ModelRegistry:
    """
    Простой реестр моделей для версионирования.
    """
    def __init__(self, registry_path='model_registry.json'):
        self.registry_path = registry_path
        self.registry = self._load_registry()
    
    def _load_registry(self):
        try:
            with open(self.registry_path, 'r') as f:
                return json.load(f)
        except FileNotFoundError:
            return {'models': [], 'current_production': None}
    
    def _save_registry(self):
        with open(self.registry_path, 'w') as f:
            json.dump(self.registry, f, indent=2, default=str)
    
    def register_model(self, model_name, model_path, metrics, description=''):
        version = len([m for m in self.registry['models'] if m['name'] == model_name]) + 1
        
        model_info = {
            'name': model_name,
            'version': version,
            'path': model_path,
            'metrics': metrics,
            'description': description,
            'registered_at': datetime.now().isoformat(),
            'status': 'staging'
        }
        
        self.registry['models'].append(model_info)
        self._save_registry()
        
        print(f'Модель {model_name} v{version} зарегистрирована')
        return model_info
    
    def promote_to_production(self, model_name, version):
        for model in self.registry['models']:
            if model['name'] == model_name and model['version'] == version:
                model['status'] = 'production'
                self.registry['current_production'] = f'{model_name}_v{version}'
                self._save_registry()
                print(f'{model_name} v{version} promoted to production')
                return
        print('Модель не найдена')
    
    def list_models(self):
        return pd.DataFrame(self.registry['models'])

# Пример использования
registry = ModelRegistry()

# Регистрируем текущую модель
registry.register_model(
    model_name='lightgbm_classifier',
    model_path='models/lightgbm.joblib',
    metrics={'accuracy': 0.52, 'roc_auc': 0.53},
    description='LightGBM baseline for direction prediction'
)

registry.promote_to_production('lightgbm_classifier', 1)

print('\nЗарегистрированные модели:')
registry.list_models()

## 3. Data Drift Detection

In [ ]:
class DriftDetector:
    """
    Детектор дрифта данных.
    """
    def __init__(self, reference_data, feature_names):
        self.reference_data = reference_data
        self.feature_names = feature_names
        self.reference_stats = self._calculate_stats(reference_data)
    
    def _calculate_stats(self, data):
        return {
            'mean': data.mean(axis=0),
            'std': data.std(axis=0),
            'median': np.median(data, axis=0)
        }
    
    def detect_drift(self, new_data, threshold=0.05):
        """
        Детекция дрифта с помощью KS-теста.
        """
        drift_results = []
        
        for i, feature in enumerate(self.feature_names):
            ref_col = self.reference_data[:, i]
            new_col = new_data[:, i]
            
            # KS test
            ks_stat, p_value = stats.ks_2samp(ref_col, new_col)
            
            drift_results.append({
                'feature': feature,
                'ks_statistic': ks_stat,
                'p_value': p_value,
                'drift_detected': p_value < threshold
            })
        
        return pd.DataFrame(drift_results)

# Загружаем данные
data_dir = 'data'
df = pd.read_parquet(f'{data_dir}/processed_data.parquet')
with open(f'{data_dir}/feature_sets.json', 'r') as f:
    feature_sets = json.load(f)

feature_cols = [f for f in feature_sets['extended_features'] if f in df.columns]

# Reference: train data, New: test data
df = df.sort_values('date')
train_data = df.iloc[:int(len(df)*0.6)][feature_cols].values
test_data = df.iloc[int(len(df)*0.8):][feature_cols].values

# Детекция дрифта
detector = DriftDetector(train_data, feature_cols)
drift_results = detector.detect_drift(test_data)

print('Результаты детекции дрифта:\n')
drifted = drift_results[drift_results['drift_detected']]
print(f'Признаков с дрифтом: {len(drifted)} из {len(feature_cols)}')
if len(drifted) > 0:
    print('\nПризнаки с дрифтом:')
    print(drifted[['feature', 'ks_statistic', 'p_value']].to_string(index=False))

In [ ]:
# Визуализация дрифта
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# KS statistics по признакам
sorted_results = drift_results.sort_values('ks_statistic', ascending=False).head(10)
colors = ['red' if d else 'green' for d in sorted_results['drift_detected']]
axes[0].barh(range(len(sorted_results)), sorted_results['ks_statistic'], color=colors)
axes[0].set_yticks(range(len(sorted_results)))
axes[0].set_yticklabels(sorted_results['feature'])
axes[0].set_xlabel('KS Statistic')
axes[0].set_title('Top-10 Features by Drift')
axes[0].axvline(x=0.1, color='black', linestyle='--', alpha=0.5)
axes[0].invert_yaxis()

# Distribution comparison для топ признака
top_feature_idx = feature_cols.index(sorted_results['feature'].iloc[0])
axes[1].hist(train_data[:, top_feature_idx], bins=50, alpha=0.5, label='Train', density=True)
axes[1].hist(test_data[:, top_feature_idx], bins=50, alpha=0.5, label='Test', density=True)
axes[1].set_xlabel(sorted_results['feature'].iloc[0])
axes[1].set_ylabel('Density')
axes[1].set_title(f'Distribution Comparison: {sorted_results["feature"].iloc[0]}')
axes[1].legend()

plt.tight_layout()
plt.show()

## 4. Monitoring Dashboard (Concept)

In [ ]:
# Мониторинг метрик
class ModelMonitor:
    def __init__(self):
        self.metrics_history = []
    
    def log_prediction(self, prediction, actual=None, latency_ms=None):
        record = {
            'timestamp': datetime.now().isoformat(),
            'prediction': prediction,
            'actual': actual,
            'latency_ms': latency_ms
        }
        self.metrics_history.append(record)
    
    def get_summary(self, last_n=100):
        recent = self.metrics_history[-last_n:] if len(self.metrics_history) > last_n else self.metrics_history
        df = pd.DataFrame(recent)
        
        summary = {
            'total_predictions': len(df),
            'avg_latency_ms': df['latency_ms'].mean() if 'latency_ms' in df else None
        }
        
        if 'actual' in df and df['actual'].notna().any():
            correct = (df['prediction'] > 0.5) == df['actual']
            summary['accuracy'] = correct.mean()
        
        return summary

# Симуляция мониторинга
monitor = ModelMonitor()

# Симулируем прогнозы
np.random.seed(42)
for _ in range(100):
    pred = np.random.uniform(0.3, 0.7)
    actual = np.random.choice([0, 1])
    latency = np.random.uniform(5, 50)
    monitor.log_prediction(pred, actual, latency)

summary = monitor.get_summary()
print('Monitoring Summary:')
for key, value in summary.items():
    if value is not None:
        print(f'  {key}: {value:.3f}' if isinstance(value, float) else f'  {key}: {value}')

## 5. CI/CD Pipeline (Concept)

```yaml
# .github/workflows/ml_pipeline.yml
name: ML Pipeline

on:
  push:
    branches: [main]
  schedule:
    - cron: '0 0 * * 0'  # Weekly retraining

jobs:
  train:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v2
      - name: Setup Python
        uses: actions/setup-python@v2
      - name: Install dependencies
        run: pip install -r requirements.txt
      - name: Run training
        run: python train.py
      - name: Run tests
        run: pytest tests/
      - name: Check drift
        run: python check_drift.py
      - name: Deploy if passed
        if: success()
        run: python deploy.py
```

## Итоги Проекта

### Что мы создали:

1. **Data Pipeline** - генерация и обработка данных
2. **Feature Engineering** - 40+ технических индикаторов
3. **ML Models** - от Logistic Regression до TFT
4. **XAI** - SHAP, Permutation Importance
5. **Backtesting** - оценка торговых стратегий
6. **Production** - API, Registry, Monitoring

### Ключевые выводы:

- Предсказание направления цен - сложная задача (~52% accuracy)
- Сложные модели не всегда лучше простых
- Интерпретируемость важна для трейдинга
- Production требует мониторинга и переобучения

### Дальнейшее развитие:

- Добавить альтернативные данные (новости, sentiment)
- Улучшить risk management
- Реализовать ensemble моделей
- Добавить A/B тестирование стратегий